In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
dataset = 2

22/01/21 12:28:40 WARN Utils: Your hostname, Rose resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface enp3s0)
22/01/21 12:28:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/21 12:28:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/21 12:28:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.sql import functions as F
# if the option is set as 1 load bank dataset
if dataset == 1:
    filename = "bank-full.csv"
    target_variable_name = 'y'
    df = spark.read.csv(filename, header=True, inferSchema=True, sep=';')
    df = df.withColumn(target_variable_name, F.when(df[target_variable_name] == 'no', 0).otherwise(1))
else:
    filename = "melb_data.csv"
    target_variable_name = "type"
    df = spark.read.csv(filename, header=True, inferSchema=True, sep=',')
    df = df.withColumn(target_variable_name, F.when(df[target_variable_name] == 'h', 0).otherwise(1))

df.show()

+---+----------+-------------------+-----+----+---------+------+------------+----------+--------+--------+--------+--------+----+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|_c0|    Suburb|            Address|Rooms|type|    Price|Method|     SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom| Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+---+----------+-------------------+-----+----+---------+------+------------+----------+--------+--------+--------+--------+----+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|  1|Abbotsford|       85 Turner St|    2|   0|1480000.0|     S|      Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0| 1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|  2|Abbotsford|    25 Bloomburg St|    2|   0|1035000.0|     S|      Biggin| 4/

22/01/21 12:29:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Suburb, Address, Rooms, Type, Price, Method, SellerG, Date, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea, YearBuilt, CouncilArea, Lattitude, Longtitude, Regionname, Propertycount
 Schema: _c0, Suburb, Address, Rooms, Type, Price, Method, SellerG, Date, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea, YearBuilt, CouncilArea, Lattitude, Longtitude, Regionname, Propertycount
Expected: _c0 but found: 
CSV file: file:///home/manhcuong/Temp/Applied_DS_Using_PySpark/chap_04/melb_data.csv


In [8]:
df.groupby(target_variable_name).count().show()

def variable_type(df):
    vars_list = df.dtypes
    char_vars = []
    num_vars = []
    for i in vars_list:
        if i[1] in ('string'):
            char_vars.append(i[0])
        else:
            num_vars.append(i[0])

    return char_vars, num_vars

+----+-----+
|type|count|
+----+-----+
|   1| 6301|
|   0|12095|
+----+-----+



In [9]:
char_vars, num_vars = variable_type(df)

In [10]:
if dataset != 1:
    char_vars.remove('Address')
    char_vars.remove('SellerG')
    char_vars.remove('Date')
    char_vars.remove('Suburb')
    
num_vars.remove(target_variable_name)
final_vars = char_vars + num_vars

In [11]:
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
from pyspark.ml.feature import QuantileDiscretizer, VectorAssembler
import scipy.stats.stats as stats